In [1]:
# https://www.kaggle.com/tunguz/bow-meta-text-and-dense-features-lgbm-clone?scriptVersionId=3540839

# Models Packages
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import time, gc
import pandas as pd
import numpy as np
from sklearn import preprocessing
from nltk.corpus import stopwords 
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import matplotlib.pyplot as plt
import pymorphy2
import nltk, re
from nltk.tokenize import ToktokTokenizer
from multiprocessing import cpu_count, Pool

In [2]:
#path = '../input/'
path = "/home/darragh/avito/data/"
path = '/Users/dhanley2/Documents/avito/data/'
path = '/home/ubuntu/avito/data/'
start_time = time.time()
full = False

print('[{}] Load Train/Test'.format(time.time() - start_time))
traindf = pd.read_csv(path + 'train.csv.zip', index_col = "item_id", parse_dates = ["activation_date"], compression = 'zip')
traindex = traindf.index
testdf = pd.read_csv(path + 'test.csv.zip', index_col = "item_id", parse_dates = ["activation_date"])
testdex = testdf.index
y = traindf.deal_probability.copy()
traindf.drop("deal_probability",axis=1, inplace=True)
print('Train shape: {} Rows, {} Columns'.format(*traindf.shape))
print('Test shape: {} Rows, {} Columns'.format(*testdf.shape))
traindf['activation_date'].value_counts()

[6.4849853515625e-05] Load Train/Test
Train shape: 1503424 Rows, 16 Columns
Test shape: 508438 Rows, 16 Columns


2017-03-20    115190
2017-03-27    114863
2017-03-19    114416
2017-03-26    113513
2017-03-28    112885
2017-03-21    110535
2017-03-22    109813
2017-03-15    108615
2017-03-23    106544
2017-03-16    106168
2017-03-17     98773
2017-03-18     97554
2017-03-24     97351
2017-03-25     97104
2017-03-29        87
2017-04-01         3
2017-03-30         3
2017-04-02         3
2017-04-03         2
2017-04-07         1
2017-03-31         1
Name: activation_date, dtype: int64

In [4]:
print('[{}] Create Validation Index'.format(time.time() - start_time))
if full:
    trnidx = (traindf.activation_date<=pd.to_datetime('2017-03-28')).values
    validx = (traindf.activation_date>=pd.to_datetime('2017-03-29')).values
else:
    trnidx = (traindf.activation_date<=pd.to_datetime('2017-03-26')).values
    validx = (traindf.activation_date>=pd.to_datetime('2017-03-27')).values

print('[{}] Combine Train and Test'.format(time.time() - start_time))
df = pd.concat([traindf,testdf],axis=0)
del traindf,testdf
gc.collect()
df['idx'] = range(df.shape[0])
print('\nAll Data shape: {} Rows, {} Columns'.format(*df.shape))

#print('[{}] Count NA row wise'.format(time.time() - start_time))
#df['NA_count_rows'] = df.isnull().sum(axis=1)

[137.64466619491577] Create Validation Index
[137.6514070034027] Combine Train and Test

All Data shape: 2011862 Rows, 17 Columns


In [6]:
lbl = preprocessing.LabelEncoder()
pcatidx = lbl.fit_transform(df["parent_category_name"].astype(str))
pd.Series(pcatidx).value_counts()

4    914200
2    243733
0    231290
5    210577
8    117282
6    109792
7     88004
3     72446
1     24538
dtype: int64

In [7]:
print('[{}] Load meta image engineered features'.format(time.time() - start_time))
featimgmeta = pd.concat([pd.read_csv(path + '../features/img_features_%s.csv.gz'%(i)) for i in range(6)])
featimgmeta.rename(columns = {'name':'image'}, inplace = True)
featimgmeta['image'] = featimgmeta['image'].str.replace('.jpg', '')
df = df.reset_index('item_id').merge(featimgmeta, on = ['image'], how = 'left').set_index('item_id')
for col in featimgmeta.columns.values[1:]:
    df[col].fillna(-1, inplace = True)
    df[col].astype(np.float32, inplace = True)
    
print('[{}] Load translated image engineered features'.format(time.time() - start_time))
feattrlten = pd.concat([pd.read_csv(path + '../features/translate_trn_en.csv.gz', compression = 'gzip'),
                        pd.read_csv(path + '../features/translate_tst_en.csv.gz', compression = 'gzip')])
# feattrlten = pd.concat([pd.read_pickle(path + '../features/translate_trn_en.pkl'),
#                        pd.read_pickle(path + '../features/translate_tst_en.pkl')])
feattrlten.fillna('', inplace = True)
feattrlten['translation'] = feattrlten['title_translated'] + ' ' + feattrlten['param_1_translated'] + ' ' \
            + feattrlten['param_2_translated'] + ' ' + feattrlten['param_3_translated'] + ' '  \
            + feattrlten['category_name_translated'] + ' ' + feattrlten['parent_category_name_translated']
feattrlten = feattrlten.set_index('item_id')[['translation']]
feattrlten.head()
df = pd.merge(df, feattrlten, left_index=True, right_index=True, how='left')
del feattrlten
gc.collect()
 
print('[{}] Load other engineered features'.format(time.time() - start_time))
featlatlon = pd.read_csv(path + '../features/avito_region_city_features.csv') # https://www.kaggle.com/frankherfert/region-and-city-details-with-lat-lon-and-clusters
featlatlon.drop(['city_region', 'city_region_id', 'region_id'], 1, inplace = True)
featpop    = pd.read_csv(path + '../features/city_population_wiki_v3.csv') # https://www.kaggle.com/stecasasso/russian-city-population-from-wikipedia/comments
featusrttl = pd.read_csv(path + '../features/user_agg.csv.gz', compression = 'gzip') # created with features/make/user_actagg_1705.py
featusrcat = pd.read_csv(path + '../features/usercat_agg.csv.gz', compression = 'gzip') # created with features/make/user_actagg_1705.py
featusrprd = pd.read_csv(path + '../features/user_activ_period_stats.gz', compression = 'gzip') # created with features/make/user_actagg_1705.py
featrdgtxt = pd.read_csv(path + '../features/ridgeText5CV.csv.gz', compression = 'gzip') # created with features/make/user_actagg_1705.py
#featrdgtxts = pd.read_csv(path + '../features/ridgeTextStr5CV.csv.gz', compression = 'gzip') # created with features/make/user_actagg_1705.py
featrdgimg = pd.read_csv(path + '../features/ridgeImg5CV.csv.gz', compression = 'gzip') # created with features/make/user_actagg_1705.py
#featrdgprc = pd.read_csv(path + '../features/price_category_ratios.gz', compression = 'gzip') # created with features/make/user_actagg_1705.py
featrdgprc = pd.read_csv(path + '../features/price_seq_category_ratios.gz', compression = 'gzip') # created with features/make/user_actagg_1705.py
featrdgprc.fillna(-1, inplace = True)
featrdgrnk = pd.read_csv(path + '../features/price_rank_ratios0906.gz', compression = 'gzip') # created with R script and stemmer
featrdgrnk.isnull().sum()
featnumf = pd.read_csv(path + '../features/numericFeats.gz', compression = 'gzip') 
featnumf.fillna(0, inplace = True)
featencfst = pd.read_csv(path + '../features/alldf_bayes_fest_1206.gz', compression = 'gzip')
featprtfst = pd.read_csv(path + '../features/pratios_fest_1206.gz', compression = 'gzip')

featprmenc = pd.read_csv(path + '../features/alldf_bayes_mean_param_1006.gz', compression = 'gzip') 
featprmtro = pd.read_csv(path + '../features/price_param_ratios1006.gz', compression = 'gzip') 

featimgprc = pd.read_csv(path + '../features/price_imagetop1_ratios.gz', compression = 'gzip') # created with features/make/priceImgRatios2705.R
featenc = pd.read_csv(path + '../features/alldf_bayes_mean.gz', compression = 'gzip') # created with features/make/user_actagg_1705.py
featct  = pd.read_csv(path + '../features/alldf_count.gz', compression = 'gzip') # created with features/make/user_actagg_1705.py
featusrttl.rename(columns={'title': 'all_titles'}, inplace = True)
df = df.reset_index().merge(featpop, on = 'city', how = 'left')
df = df.merge(featlatlon, on = ['city', 'region'], how = 'left')
df['population'].fillna(-1, inplace = True)
df = df.set_index('item_id')
keep = ['user_id', 'all_titles', 'user_avg_price', 'user_ad_ct']
df = df.reset_index().merge(featusrttl[keep], on = 'user_id').set_index('item_id')
keep = ['user_id', 'parent_category_name', 'usercat_avg_price', 'usercat_ad_ct']
gc.collect()
df = df.reset_index().merge(featusrcat[keep], on = ['user_id', 'parent_category_name']).set_index('item_id')
keep = ['user_id', 'user_activ_sum', 'user_activ_mean', 'user_activ_var']
gc.collect()
df = df.reset_index().merge(featusrprd[keep], on = ['user_id'], how = 'left').set_index('item_id')
print('\nAll Data shape: {} Rows, {} Columns'.format(*df.shape))  

[192.95045518875122] Load meta image engineered features
[212.38360619544983] Load translated image engineered features
[240.86302661895752] Load other engineered features

All Data shape: 2011862 Rows, 44 Columns


In [8]:
print('[{}] Resort data correctly'.format(time.time() - start_time))
df.sort_values('idx', inplace = True)
df.drop(['idx'], axis=1,inplace=True)
df.reset_index(inplace = True)
df.head()
df = pd.concat([df.reset_index(),featenc, featct, featrdgtxt, featrdgprc, featimgprc, featrdgrnk, featnumf, featprmenc, featprmtro, featencfst, featprtfst],axis=1)
#df['ridge_txt'] = featrdgtxt['ridge_preds'].values
#df = pd.concat([df.reset_index(),featenc, featct, ],axis=1)

print('[{}] Create folds'.format(time.time() - start_time))
foldls = [["2017-03-15", "2017-03-16", "2017-03-17"], \
       ["2017-03-18", "2017-03-19", "2017-03-20"], \
       ["2017-03-21", "2017-03-22", "2017-03-23"], \
       ["2017-03-24", "2017-03-25", "2017-03-26"], \
        ["2017-03-27", "2017-03-28", "2017-03-29", \
            "2017-03-30", "2017-03-31", "2017-04-01", \
            "2017-04-02", "2017-04-03","2017-04-07"]]
foldls = [[pd.to_datetime(d) for d in f] for f in foldls]
df['fold'] = -1
for t, fold in enumerate(foldls):
    df['fold'][df.activation_date.isin(fold)] = t
df['fold'].value_counts()
df.head()



df['ridge_img'] = featrdgimg['ridge_img_preds'].values
df = df.set_index('item_id')
df.drop(['index'], axis=1,inplace=True)
df.columns
del featusrttl, featusrcat, featusrprd, featenc, featrdgprc, featimgprc, featnumf, featprmenc, featprmtro, featencfst, featprtfst
# del featusrttl, featusrcat, featusrprd, featenc, featrdgtxts
gc.collect()

print('[{}] Feature Engineering'.format(time.time() - start_time))
for col in df.columns:
    if 'price' in col:
        print(f'Fill {col}')
        df[col].fillna(-999,inplace=True)

for col in df.columns:
    if 'user_activ' in col:
        print(f'fill {col}')
        df[col].fillna(-9,inplace=True)
df["image_top_1"].fillna(-999,inplace=True)

del featct, featlatlon, featimgmeta, featpop, featrdgimg, featrdgtxt
gc.collect()

print('[{}] Manage Memory'.format(time.time() - start_time))
for col in df.columns:
    if np.float64 == df[col].dtype:
        df[col] = df[col].astype(np.float32)
    if np.int64 == df[col].dtype:
        df[col] = df[col].astype(np.int32)
    gc.collect()
df.dtypes

[372.74321365356445] Resort data correctly
[378.13086771965027] Create folds


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


[387.6946907043457] Feature Engineering
Fill price
Fill user_avg_price
Fill usercat_avg_price
Fill pcat_price_rratio
Fill cat_price_rratio
Fill ttl_price_rratio
Fill dscr_price_rratio
Fill pcat_log_price_rratio
Fill user_log_price_rratio
Fill cat_price_iratio
Fill reg_price_iratio
Fill reg_price_gratio
Fill cty_price_gratio
Fill ttlst_price_rratio
Fill ttlst_city_price_rratio
Fill ttlst_prm_price_rratio
Fill par1cty_price_prratio
Fill par2cty_price_prratio
Fill par1utyp_price_prratio
Fill par2utyp_price_prratio
fill user_activ_sum
fill user_activ_mean
fill user_activ_var
[387.8854446411133] Manage Memory


user_id                                                                                                   object
region                                                                                                    object
city                                                                                                      object
parent_category_name                                                                                      object
category_name                                                                                             object
param_1                                                                                                   object
param_2                                                                                                   object
param_3                                                                                                   object
title                                                                                           

In [9]:
print('[{}] Text Features'.format(time.time() - start_time))
df['text_feat'] = df.apply(lambda row: ' '.join([
    str(row['param_1']), 
    str(row['param_2']), 
    str(row['param_3'])]),axis=1) # Group Param Features
df.drop(["param_1","param_2","param_3"],axis=1,inplace=True)

print('[{}] Text Features'.format(time.time() - start_time))
df['description'].fillna('unknowndescription', inplace=True)
df['title'].fillna('unknowntitle', inplace=True)
df['text']      = (df['description'].fillna('') + ' ' + df['title'] + ' ' + 
  df['parent_category_name'].fillna('').astype(str) + ' ' + df['category_name'].fillna('').astype(str) )

print('[{}] Create Time Variables'.format(time.time() - start_time))
df["Weekday"] = df['activation_date'].dt.weekday
df.drop(["activation_date","image"],axis=1,inplace=True)

print('[{}] Make Item Seq number as contiuous also'.format(time.time() - start_time))
df["item_seq_number_cont"] = df["item_seq_number"]
df['city'] = df['region'].fillna('').astype(str) + '_' + df['city'].fillna('').astype(str)
df.columns
print('[{}] Encode Variables'.format(time.time() - start_time))
df.drop(['user_id'], 1, inplace = True)
categorical = ["region","parent_category_name","user_type", 'city', 'category_name', "item_seq_number", 'image_top_1']
print("Encoding :",categorical)
# Encoder:
lbl = preprocessing.LabelEncoder()
for col in categorical:
    df[col] = lbl.fit_transform(df[col].astype(str))
  
print('[{}] Meta Text Features'.format(time.time() - start_time))
textfeats = ["description","text_feat", "title"]
for cols in textfeats:
    df[cols] = df[cols].astype(str) 
    df[cols] = df[cols].astype(str).fillna('nicapotato') # FILL NA
    df[cols] = df[cols].str.lower() # Lowercase all text, so that capitalized words dont get treated differently
    df[cols + '_num_chars'] = df[cols].apply(len) # Count number of Characters
    df[cols + '_num_words'] = df[cols].apply(lambda comment: len(comment.split())) # Count number of Words
    df[cols + '_num_unique_words'] = df[cols].apply(lambda comment: len(set(w for w in comment.split())))
    df[cols + '_words_vs_unique'] = df[cols+'_num_unique_words'] / df[cols+'_num_words'] * 100 # Count Unique Words
    gc.collect()
df.info()
for cols in ['translation']:
    df[cols] = df[cols].astype(str) 
    df[cols] = df[cols].astype(str).fillna('nicapotato') # FILL NA
    df[cols] = df[cols].str.lower() # Lowercase all text, so that capitalized words dont get treated differently

    
print('[{}] Manage Memory'.format(time.time() - start_time))
for col in df.columns:
    if np.float64 == df[col].dtype:
        df[col] = df[col].astype(np.float32)
    if np.int64 == df[col].dtype:
        df[col] = df[col].astype(np.int32)
    gc.collect()
df.info()

[441.89318895339966] Text Features
[735.6180574893951] Text Features
[741.4385895729065] Create Time Variables
[743.6199576854706] Make Item Seq number as contiuous also
[744.8901000022888] Encode Variables
Encoding : ['region', 'parent_category_name', 'user_type', 'city', 'category_name', 'item_seq_number', 'image_top_1']
[778.3553726673126] Meta Text Features
<class 'pandas.core.frame.DataFrame'>
Index: 2011862 entries, b912c3c6a6ad to d374d332992f
Columns: 189 entries, region to title_words_vs_unique
dtypes: float32(99), float64(3), int32(64), int64(17), object(6)
memory usage: 1.6+ GB
[828.5959520339966] Manage Memory
<class 'pandas.core.frame.DataFrame'>
Index: 2011862 entries, b912c3c6a6ad to d374d332992f
Columns: 189 entries, region to title_words_vs_unique
dtypes: float32(102), int32(81), object(6)
memory usage: 1.5+ GB


In [10]:
print('[{}] Clean text and tokenize'.format(time.time() - start_time))
toktok = ToktokTokenizer()
tokSentMap = {}
morpher = pymorphy2.MorphAnalyzer()
def tokSent(sent):
    sent = sent.replace('/', ' ')
    return " ".join(morpher.parse(word)[0].normal_form for word in toktok.tokenize(rgx.sub(' ', sent)))
def tokCol(var):
    return [tokSent(s) for s in var.tolist()]
rgx = re.compile('[%s]' % '!"#%&()*,-./:;<=>?@[\\]^_`{|}~\t\n')   

partitions = 4 
def parallelize(data, func):
    data_split = np.array_split(data.values, partitions)
    pool = Pool(partitions)
    data = pd.concat([pd.Series(l) for l in pool.map(tokCol, data_split)]).values
    pool.close()
    pool.join()
    return data

load_text = True
text_cols = ['description', 'text', 'text_feat', 'title', 'translation']
if load_text:
    dftxt = pd.read_csv(path + '../features/text_features_morphed.csv.gz', compression = 'gzip')
    for col in text_cols:
        print(col + ' load tokenised [{}]'.format(time.time() - start_time))
        df[col] = dftxt[col].values
        df.fillna(' ', inplace = True)
    del dftxt
else:
    for col in text_cols:
        print(col + ' tokenise [{}]'.format(time.time() - start_time))
        df[col] = parallelize(df[col], tokCol)
    df[text_cols].to_csv(path + '../features/text_features_morphed.csv.gz', compression = 'gzip')
gc.collect()

print('[{}] Finished tokenizing text...'.format(time.time() - start_time))
df.head()

[837.6263468265533] Clean text and tokenize
description load tokenised [872.8003618717194]
text load tokenised [874.8688545227051]
text_feat load tokenised [876.9606008529663]
title load tokenised [878.7956805229187]
translation load tokenised [880.7158381938934]
[882.8781893253326] Finished tokenizing text...


,region,city,parent_category_name,category_name,title,description,price,item_seq_number,user_type,image_top_1,...,description_num_unique_words,description_words_vs_unique,text_feat_num_chars,text_feat_num_words,text_feat_num_unique_words,text_feat_words_vs_unique,title_num_chars,title_num_words,title_num_unique_words,title_words_vs_unique
item_id,,,,,,,,,,,,,,,,,,,,,
b912c3c6a6ad,19,1313,4,42,кокоби кокон для сон,кокон для сон малыш пользоваться маленький мес...,400.0,9108,1,13,...,7,100.0,33,4,3,75.000000,21,3,3,100.0
2dac0150717d,17,1233,2,22,стойка для одежда,стойка для одежда под вешалка с бутик,3000.0,8389,1,2723,...,7,100.0,14,3,2,66.666664,17,3,3,100.0
ba83aefab5dc,16,1172,0,2,philips bluray,в хороший состояние домашний кинотеатр с blu r...,4000.0,32856,1,2260,...,17,100.0,35,7,6,85.714287,14,2,2,100.0
02996f1dd2ea,21,1557,4,42,автокресло,продать кресло от0 25кг,2200.0,14637,0,2838,...,3,100.0,28,4,3,75.000000,10,1,1,100.0
7c90be56d2ab,4,250,6,0,ваза 2110 2003,весь вопрос по телефон,40000.0,15299,1,1408,...,4,100.0,26,5,5,100.000000,14,3,3,100.0


In [11]:
print('[{}] [TF-IDF] Term Frequency Inverse Document Frequency Stage'.format(time.time() - start_time))
russian_stop = set(stopwords.words('russian'))
tfidf_para = {
    "stop_words": russian_stop,
    "token_pattern": r'\w{1,}',
    "sublinear_tf": True,
    "dtype": np.float32,
    "smooth_idf":False
}
countv_para = {
    "stop_words": russian_stop,
    "analyzer": 'word',
    "token_pattern": r'\w{1,}',
    "lowercase": True,
    "min_df": 10
}
def get_col(col_name): return lambda x: x[col_name]
vectorizer = FeatureUnion([
        ('text',TfidfVectorizer(
            ngram_range=(1, 2),
            max_features=50000,
            **tfidf_para,
            preprocessor=get_col('text'))),
        ('text_feat',CountVectorizer(
            **countv_para,
            preprocessor=get_col('text_feat'))),
        ('title',CountVectorizer(
            **countv_para,
            preprocessor=get_col('title'))),
    ])
    
start_vect=time.time()
vectorizer.fit(df.loc[traindex,:].to_dict('records'))
ready_df = vectorizer.transform(df.to_dict('records'))
tfvocab = vectorizer.get_feature_names()
tfvocab[:50]
print('[{}] Vectorisation completed'.format(time.time() - start_time))
# Drop Text Cols
df.drop(textfeats+['text', 'all_titles', 'translation'], axis=1,inplace=True)
gc.collect()

[882.9482765197754] [TF-IDF] Term Frequency Inverse Document Frequency Stage
[2079.12335395813] Vectorisation completed


6

In [ ]:
print('[{}] Drop all the categorical'.format(time.time() - start_time))
df.drop(categorical, axis=1,inplace=True)

In [12]:
# Training and Validation Set
lgbm_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective' : 'regression',
    'metric' : 'rmse',
    'num_leaves' : 500,
    'nthread': 16,
    'learning_rate' : 0.02,
    'feature_fraction' : 0.5,
    'verbosity' : 0,
    'seed'      : 11
}    

[2080.178646802902] Drop all the categorical


In [ ]:
#4    914200
#2    243733
#0    231290
#5    210577
#8    117282
#6    109792
#7     88004
#3     72446
#1     24538

In [13]:
pcatids = pd.Series(pcatidx, index=df.index)

In [ ]:
# Placeholder for predictions
df['fold'].value_counts()
y_pred_trn = pd.Series(-np.zeros(df.loc[traindex,:].shape[0]), index = traindex)
y_pred_tst = pd.Series(-np.zeros(df.loc[testdex ,:].shape[0]), index = testdex)

for f in range(6):
    print('Fold %s'%(f) + ' [{}] Modeling Stage'.format(time.time() - start_time))
    trnidx = (df['fold'].loc[traindex] != f).values
    X_train = csr_matrix(hstack([csr_matrix(df.drop('fold', 1).loc[traindex,:][trnidx].values),ready_df[0:traindex.shape[0]][trnidx]]))
    y_train = y[trnidx]

    pcattrn = pcatids.loc[traindex][trnidx].values
    # 5 is the test fold
    if f == 5:
        X_test = csr_matrix(hstack([csr_matrix(df.drop('fold', 1).loc[testdex,:].values),ready_df[traindex.shape[0]:]]))
        pcattst = pcatids.loc[testdex].values
    else:
        X_test = csr_matrix(hstack([csr_matrix(df.drop('fold', 1).loc[traindex,:][~trnidx].values),ready_df[0:traindex.shape[0]][~trnidx]]))
        pcattst = pcatids.loc[traindex][~trnidx].values
        y_test  = y[~trnidx]
    tfvocab = df.drop('fold', 1).columns.tolist() + vectorizer.get_feature_names()
    for shape in [X_train, X_test]:
        print("Fold {} : {} Rows and {} Cols".format(f, *shape.shape))
    gc.collect();
    pred = np.zeros(pcattst.shape[0])
    
    # Run a model on everiy parent category
    for p in range(9):
        # LGBM Dataset Formatting 
        print('Fold %s Pcat %s'%(f, p) + ' [{}]'.format(time.time() - start_time))

        lgtrain = lgb.Dataset(X_train[pcattrn==p], y_train[pcattrn==p],
                            feature_name=tfvocab)
        lgb_clf = lgb.train(
            lgbm_params,
            lgtrain,    
            num_boost_round = 900,
            verbose_eval=2000)   
        pred[pcattst==p] = lgb_clf.predict(X_test[pcattst==p])
        del lgtrain
        gc.collect()
        if f != 5:
            print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test[pcattst==p], pred[pcattst==p])))
        
    print("Model Evaluation Stage")
    if f == 5:
        y_pred_tst[:] = pred
    else:
        y_pred_trn[~trnidx] = pred
        print('RMSE Full Fold:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_trn[~trnidx])))
        
    ## LGBM Dataset Formatting 
    #lgtrain = lgb.Dataset(X_train, y_train,
    #                feature_name=tfvocab)
    del X_train, y_train
    gc.collect()

    #lgb_clf = lgb.train(
    #    lgbm_params,
    #    lgtrain,    
    #    num_boost_round = 1200,
    #    verbose_eval=200)    

    #print("Model Evaluation Stage")
    #if f == 5:
    #    y_pred_tst[:] = lgb_clf.predict(X_test)
    #else:
    #    y_pred_trn[~trnidx] = lgb_clf.predict(X_test)
    #    print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_trn[~trnidx])))
    del X_test
    gc.collect()
    y_pred_trn.to_csv("lgbCV_2406_trn.csv",index=True)
    y_pred_tst.to_csv("lgbCV_2406_tst.csv",index=True)    

Fold 0 [38533.116355895996] Modeling Stage
Fold 0 : 1189868 Rows and 71734 Cols
Fold 0 : 313556 Rows and 71734 Cols
Fold 0 Pcat 0 [38743.51289272308]
RMSE: 0.2455372859991009
Fold 0 Pcat 1 [40444.14125084877]
RMSE: 0.1895566877604732
Fold 0 Pcat 2 [40577.34098815918]


In [ ]:
lgsub = pd.concat([y_pred_trn, y_pred_tst]).reset_index()
lgsub.rename(columns = {0 : 'deal_probability'}, inplace=True)
#lgsub['deal_probability'].clip(0.0, 1.0, inplace=True)
lgsub.set_index('item_id', inplace = True)
print('RMSE for all :', np.sqrt(metrics.mean_squared_error(y, lgsub.loc[traindex])))
# RMSE for all : 0.2168
lgsub.to_csv("lgCV_2406.csv.gz",index=True,header=True, compression = 'gzip')

lgsub.to_csv(path + "../sub/lgCV_2406.csv.gz",index=True,header=True, compression = 'gzip')